In [1]:
# №1

iter = 100 # кол-во тераций
x_old = 0
a = 0.1 # шаг сходиомсти
ee = 1e-4
         
for i in range(iter):
    x_new = x_old - a * (2*x_old + 4)
    
    if abs(x_new - x_old) < ee:
        break
    
    x_old = x_new
    
min_value = x_old**2 + 4*x_old + 4

print(round(x_old), round(min_value))

-2 0


In [3]:
# №2.1


from scipy.optimize import linprog
# Коэффициенты целевой функции 
c = [-5, -3]
# Матрица коэффициентов ограничений (Ax <= b)
A = [[2, 1]]
# Вектор правых частей ограничений
b = [100]

x_bounds = (0, None)
y_bounds = (0, None)

result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='simplex')

print(f"Оптимальное количество товара A: {result.x[0]}")
print(f"Оптимальное количество товара B: {result.x[1]}")
print(f"Максимальная прибыль: {-result.fun}")


Оптимальное количество товара A: 0.0
Оптимальное количество товара B: 100.0
Максимальная прибыль: 300.0


C:\Users\kache\AppData\Local\Temp\ipykernel_13960\1810387974.py:15: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='simplex')


In [4]:
# №3


def knapsack(items, capacity):
    n = len(items)
    max_value = 0
    best_subset = []

    # Перебираем все возможные маски (подмножества)
    for m in range(0, 1 << n):  # 1 << n = 2^n
        current_weight = 0
        current_value = 0
        subset = []

        for i in range(n):
            if (m >> i) & 1:  # Проверяем, включён ли i-й предмет
                weight, value = items[i]
                current_weight += weight
                current_value += value
                subset.append(items[i])

        if current_weight <= capacity and current_value > max_value:
            max_value = current_value
            best_subset = subset

    return best_subset, max_value

items = [(10, 60), (20, 100), (30, 120)]
capacity = 50

selected, total = knapsack(items, capacity)

print("Оптимальный набор предметов:", selected)
print("Максимальная ценность:", total)

Оптимальный набор предметов: [(20, 100), (30, 120)]
Максимальная ценность: 220


In [6]:
# №4


import itertools

def traveling_salesman(graph):
    n = len(graph)
    cities = list(range(n))
    min_length = float('inf')
    best_path = []

    # Начинаем с города 0
    start = 0
    # Перебираем все перестановки остальных городов
    for perm in itertools.permutations(cities[1:]):
        current_path = [start] + list(perm) + [start]
        current_length = 0
        valid = True

        for i in range(len(current_path) - 1):
            a = current_path[i]
            b = current_path[i+1]
            if graph[a][b] == 0:  
                valid = False
                break
            current_length += graph[a][b]

        if valid and current_length < min_length:
            min_length = current_length
            best_path = current_path

    return best_path, min_length

# Пример использования
graph = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

path, length = traveling_salesman(graph)

print("Минимальная длина пути:", length)
print("Порядок посещения городов:", path)

Минимальная длина пути: 80
Порядок посещения городов: [0, 1, 3, 2, 0]


In [5]:
# №5

import sympy as sp

t = sp.symbols('t')
Q0 = 100 
lambda_ = 0.05  
T = 365  

# Функция дебита
Q = Q0 * sp.exp(-lambda_ * t)

integral = sp.integrate(Q, (t, 0, T))

# Вывод результата
print(f"Накопленная добыча: {integral.evalf():.2f} м³")

Накопленная добыча: 2000.00 м³
